In [1]:
from pathlib import Path
from typing import List
import pickle
import re
import os

In [1]:
def find_files_with_prefix(directory: str, prefix: str) -> List[Path]:
    """
    Finds all files in the given directory that start with the specified prefix.

    Args:
        directory (str): Path to the directory to search.
        prefix (str): The prefix that filenames should start with.

    Returns:
        List[Path]: List of Path objects for matching files.
    """
    dir_path = Path(directory)
    return [file for file in dir_path.iterdir() if file.is_file() and file.name.startswith(prefix)]


NameError: name 'List' is not defined

In [3]:
downsample_freq = 8
max_gap_annos_in_sec = 15
window_size_sec = 25
pre_thresh_sec = 60*5
post_thresh_sec = 60*3
DIR_preprocessed =  f"/home/swolf/asim_shared/preprocessed_data/downsample_freq={downsample_freq},no_windows"

config = DIR_preprocessed.split("/")[-1]
MPs_path = f"/home/swolf/asim_shared/results/MP/downsample_freq={downsample_freq},no_windows/seq_len{window_size_sec}sec"

In [4]:
mps_filenames = [filename for filename in os.listdir(MPs_path) if filename.endswith(".pkl")]
preprocessed_filenames = [filename for filename in os.listdir(DIR_preprocessed) if filename.endswith("_preprocessed.pkl")]

In [5]:
mps_list = []
label_list = []
for i, mp_filename in enumerate(mps_filenames[:1000]):
    if i%100 ==0:
        print(i)
    mp_filename=mp_filename.__str__()
    # Load MP
    with open(os.path.join(MPs_path, mp_filename), "rb") as f:
        # data = array_list
        mps_list.append(pickle.load(f)[:, 0].reshape(-1, 1))
    # print(os.path.join(DIR_preprocessed, mps_filenames[0][3:-4]+"_preprocessed.pkl"))
    with open(os.path.join(DIR_preprocessed, mp_filename[3:-4]+"_preprocessed.pkl"), "rb") as g:
        label_list.append(pickle.load(g)["channels"][0]["labels"][0])
        # label_list.append(pickle.load(g))


0


/tmp/ipykernel_2251041/1666676929.py:13: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  label_list.append(pickle.load(g)["channels"][0]["labels"][0])


100
200
300
400
500
600
700
800
900


In [6]:
# label_list[]

In [7]:
import numpy as np
test_list = []
for i, labels in enumerate(label_list):
    if np.sum(labels) > 0:
        test_list.append(i)
        # print(f"Sequence {i}: contains {np.sum(labels)} event samples")
len(test_list)

141

In [9]:
print(len(label_list)==len(mps_list))

True


In [10]:
label_list[0]

array([0, 0, 0, ..., 0, 0, 0], shape=(13264,))

In [11]:
mps_list[0]

mparray([[15.09674193645468],
         [13.825901100183],
         [15.572337997950749],
         ...,
         [13.045173944610982],
         [13.051950784690577],
         [13.050601518616684]], shape=(13065, 1), dtype=object)

In [12]:
# Add parent directory (../) to sys.path
import sys
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [13]:
from utils.metrics import compute_sensitivity_false_alarm_rate_timing_tolerance

In [14]:
from matrix_profile import MatrixProfile #compute_approx_matrix_profile #get_top_k_anomaly_indices

In [15]:
amount_of_annomalies_per_record = 10
annomaly_indices = [MatrixProfile.get_top_k_anomaly_indices(matrix_profile=mp.flatten(), k=amount_of_annomalies_per_record)
                    for mp in mps_list]

In [16]:
annomaly_indices_cons = [MatrixProfile.mean_of_all_consecutive_anomalies(indices=annos, n=3, max_gap = downsample_freq*max_gap_annos_in_sec)
                         for annos in annomaly_indices]

In [17]:
sensitivity, false_alarm_rate, true_positives, false_positives = compute_sensitivity_false_alarm_rate_timing_tolerance(
    label_sequences=label_list, detection_indices=annomaly_indices_cons, lower=pre_thresh_sec, upper=post_thresh_sec, frequency=downsample_freq
)

False alarms per hour 0.28396843098294433


In [18]:
print(f"{sensitivity=}, {false_alarm_rate=}")
print(f"{true_positives=}, {false_positives=}")

sensitivity=0.21631205673758866, false_alarm_rate=0.952
true_positives=61, false_positives=1190


In [ ]:
label_list[0].__len__()

13264